# nvImageCodec with cuPy

In [ ]:
import os
import cv2
import cupy as cp
from matplotlib import pyplot as plt

Setting resource folder

In [ ]:
resources_dir = os.getenv("PYNVIMGCODEC_EXAMPLES_RESOURCES_DIR", "../assets/images/")

Import nvImageCodec module and create both Decoder and Encoder

In [ ]:
from nvidia import nvimgcodec
decoder = nvimgcodec.Decoder()
encoder = nvimgcodec.Encoder()

Load jpeg2000 image with nvImageCodec

In [ ]:
nv_img = decoder.read(resources_dir + "cat-1046544_640.jp2")
print(nv_img.__cuda_array_interface__)

Pass nvImageCodec Image to cupy asarray as it accepts \_\_cuda_array_interface\_\_ 

In [ ]:
%%time
cp_img = cp.asarray(nv_img)
print(cp_img.__cuda_array_interface__)

Convert to numpy and show

In [ ]:
np_img4k = cp.asnumpy(cp_img)
plt.imshow(np_img4k)

Lets do some opration on image in GPU using cupyx.scipy

In [ ]:
import cupyx.scipy.ndimage

In [ ]:
cp_img_rotated = cupyx.scipy.ndimage.rotate(cp_img, 90)
cp_img_gaussian = cupyx.scipy.ndimage.gaussian_filter(cp_img, sigma = 5)

print(cp_img_rotated.__cuda_array_interface__)

In [ ]:
np_img = cp.asnumpy(cp_img_rotated)
plt.imshow(np_img)

In [ ]:
np_img = cp.asnumpy(cp_img_gaussian)
plt.imshow(np_img)

Convert cupy Image to nvImageCodec Image using \_\_cuda_array_interface\_\_

In [ ]:
%%time
nv_rotated_img = nvimgcodec.as_image(cp_img_rotated)

Check content

In [ ]:
plt.imshow(nv_rotated_img.cpu())

Save as Jpeg2000

In [ ]:
encoder.write("rotated.j2k", nv_rotated_img)

Load with OpenCv to verify

In [ ]:
image = cv2.imread("rotated.j2k")
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.imshow(image)

Save cupy image to jpg with nvImageCodec

In [ ]:
nv_img_gaussian = nvimgcodec.as_image(cp_img_gaussian)
encoder.write("gaussian.jpg", nv_img_gaussian)

Read back with OpenCV to verify

In [ ]:
image = cv2.imread("gaussian.jpg")
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.imshow(image)

## DLPack

Passing cuPy ndarray to nvImageCodec using DLPack. cuPy ndarray has both __cuda_array_interface__ and __dlpack__ interface. By default, __cuda_array_interface__ is preferred and will be used when we pass cuPy ndarray to nvImageCodec using as_image function. If we would like to use DLPack, firstly PyCapsule need to be taken from cuPy ndarray and then it needs to be passed to nvImageCodec.

In [ ]:
dlpack_img = cp_img_gaussian.toDlpack()
nv_img_gaussian = nvimgcodec.as_image(dlpack_img)
encoder.write("gaussian_dlpack.jpg", nv_img_gaussian)

In [ ]:
image = cv2.imread("gaussian_dlpack.jpg")
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.imshow(image)


Passing nvImageCodec Image to cuPy using DLPack

In [ ]:
nv_img = decoder.read(resources_dir + "cat-1046544_640.jp2")
cp_img = cp.from_dlpack(nv_img)

In [ ]:
np_img = cp.asnumpy(cp_img)
plt.imshow(np_img)

Alternatively, there is possibility to firstly take PyCapsule and then pass it to cuPy 

In [ ]:
nv_img = decoder.read(resources_dir + "cat-1046544_640.jp2")
py_cap = nv_img.__dlpack__()
cp_img = cp.from_dlpack(py_cap)

In [ ]:
np_img = cp.asnumpy(cp_img)
plt.imshow(np_img4k)